## README: This is the final notebook. Use this notebook to paste the final code.

### Pre Processing of Tweets

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta, datetime
from dateutil import parser
import re

In [20]:
tweets = pd.read_csv('raw_trump_tweets.csv')

In [21]:
# >= 20th jan 2017, <= 31st Dec 2018
tweets['date']= pd.to_datetime(tweets['date'])
tweets = tweets[tweets['date'] >= np.datetime64('2017-01-20')]
tweets = tweets[tweets['date'] <= np.datetime64('2018-12-31')]


In [22]:
def process_and_remove_duplicates(tweets, date):
    duplicated_rows = tweets[tweets['date'] == date]
    id = max(duplicated_rows['id'])
    max_favorite = max(duplicated_rows['favorites'])
    max_retweet = max(duplicated_rows['retweets'])
    combined_tweet = ""
    device = duplicated_rows['device'].iloc[0]
    for index, row in duplicated_rows.iterrows():
        combined_tweet = row['text'] + ' ' + combined_tweet.strip('.')
    tweets = tweets[tweets.date != date]
    row = {'id': id, 'text': combined_tweet, 'isRetweet': 'f', 'isDeleted': 'f', 'device': device, 'favorites': max_favorite, 'retweets': max_retweet, 'date': date}
    tweets = tweets.append(row, ignore_index = True)   
    print(f"Combined {len(duplicated_rows)} tweets on {date}")
    return tweets

In [23]:
dates = tweets["date"]
duplicated = tweets[dates.isin(dates[dates.duplicated()])]
duplicated_dates = duplicated['date'].unique()

for date in duplicated_dates:
    tweets = process_and_remove_duplicates(tweets, date)

Combined 2 tweets on 2018-11-21T22:18:00.000000000
Combined 2 tweets on 2018-09-10T21:59:00.000000000
Combined 2 tweets on 2018-08-14T01:37:00.000000000
Combined 2 tweets on 2018-05-31T10:56:00.000000000
Combined 2 tweets on 2018-02-01T22:37:00.000000000
Combined 2 tweets on 2018-03-09T17:50:00.000000000
Combined 2 tweets on 2017-12-29T12:50:00.000000000
Combined 2 tweets on 2017-12-29T12:48:00.000000000
Combined 3 tweets on 2017-12-28T22:17:00.000000000
Combined 2 tweets on 2017-12-22T20:47:00.000000000
Combined 2 tweets on 2017-12-16T03:09:00.000000000
Combined 2 tweets on 2017-11-18T13:49:00.000000000
Combined 2 tweets on 2017-11-15T10:22:00.000000000
Combined 2 tweets on 2017-07-11T10:53:00.000000000
Combined 2 tweets on 2017-07-11T10:37:00.000000000
Combined 2 tweets on 2017-10-28T21:09:00.000000000
Combined 2 tweets on 2017-10-26T01:47:00.000000000
Combined 2 tweets on 2017-10-14T11:08:00.000000000
Combined 2 tweets on 2017-05-10T10:31:00.000000000
Combined 2 tweets on 2017-09-30

Combined 2 tweets on 2018-03-07T23:13:00.000000000
Combined 2 tweets on 2018-06-30T10:59:00.000000000
Combined 2 tweets on 2018-06-30T10:49:00.000000000
Combined 2 tweets on 2018-06-28T03:24:00.000000000
Combined 2 tweets on 2018-06-25T16:58:00.000000000
Combined 2 tweets on 2018-06-23T14:00:00.000000000
Combined 2 tweets on 2018-06-23T11:21:00.000000000
Combined 3 tweets on 2018-06-23T11:19:00.000000000
Combined 2 tweets on 2018-06-23T11:16:00.000000000
Combined 2 tweets on 2018-06-21T20:46:00.000000000
Combined 4 tweets on 2018-06-19T13:52:00.000000000
Combined 2 tweets on 2018-06-18T13:50:00.000000000
Combined 3 tweets on 2018-06-17T14:36:00.000000000
Combined 2 tweets on 2018-06-15T11:56:00.000000000
Combined 4 tweets on 2018-06-15T11:55:00.000000000
Combined 2 tweets on 2018-12-06T08:53:00.000000000
Combined 2 tweets on 2018-06-14T15:09:00.000000000
Combined 2 tweets on 2018-06-14T15:08:00.000000000
Combined 2 tweets on 2018-06-13T09:40:00.000000000
Combined 2 tweets on 2018-12-06

In [24]:
## tweets that are Trump's
tweets1 = tweets[tweets['isRetweet'] == 'f'].reset_index(drop=True)
tweets1 = tweets1[tweets1['text'].str.contains('RT @') == False]
tweets1['cleaned_text'] = tweets1['text'].apply(lambda x: re.sub(r'https?:\/\/\S*', '', x, flags=re.MULTILINE))
tweets1 = tweets1[tweets1['cleaned_text'].str.strip() != '']

tweets2 = tweets1.loc[:, ['id', 'cleaned_text', 'favorites', 'retweets', 'date']]
tweets2['date_new'] = [i + timedelta(hours = 8) for i in tweets2['date']]
tweets2['date_part'] = [i.date() for i in tweets2['date_new']]
tweets2['time_part'] = [i.time() for i in tweets2['date_new']]
tweets2['hour'] = [int(str(i).split(":")[0]) for i in tweets2['time_part']]
tweets2['year'] = [int(str(i).split("-")[0]) for i in tweets2['date_part']]
tweets2['month'] = [int(str(i).split("-")[1]) for i in tweets2['date_part']]


In [25]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_mentions(string):
    return re.sub("@[A-Za-z0-9]+","",string)
    

In [26]:
tweets2["cleaned_text"] = tweets2["cleaned_text"].str.lower()
# for i in range(0,5088):
#     tweets2.loc[i,'cleaned_text2'] = remove_emoji(df.loc[i,'cleaned_text'])
# can be made into :
tweets2["cleaned_text2"] = tweets2.cleaned_text.apply(remove_emoji)
# need to test

# for i in range(0,5088):
#     df.loc[i,'cleaned_text2'] = re.sub("@[A-Za-z0-9]+","",df.loc[i,'cleaned_text2'])

tweets2.cleaned_text2 = tweets2.cleaned_text.apply(remove_mentions)

In [18]:
tweets2

,id,cleaned_text,favorites,retweets,date,date_new,date_part,time_part,hour,year,month,cleaned_text2
0,9.384230e+17,make america great again!,157963,37189,2017-06-12 15:00:00,2017-06-12 23:00:00,2017-06-12,23:00:00,23,2017,6,make america great again!
1,9.391890e+17,make america great again!,56596,11433,2017-08-12 17:46:00,2017-08-13 01:46:00,2017-08-13,01:46:00,1,2017,8,make america great again!
2,9.353400e+17,thank you rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,thank you rand!
3,9.253890e+17,thank you @luisriveramarin!,23521,4574,2017-10-31 15:48:00,2017-10-31 23:48:00,2017-10-31,23:48:00,23,2017,10,thank you @luisriveramarin!
4,8.997980e+17,"join me live from fort myer in arlington, virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,"join me live from fort myer in arlington, virg..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5249,9.862190e+17,"employment is up, taxes are down. enjoy! i am ...",106514,19661,2018-04-17 12:24:00,2018-04-17 20:24:00,2018-04-17,20:24:00,20,2018,4,"employment is up, taxes are down. enjoy! i am ..."
5252,9.791090e+17,i am pleased to announce that i intend to nomi...,66173,13399,2018-03-28 21:31:00,2018-03-29 05:31:00,2018-03-29,05:31:00,5,2018,3,i am pleased to announce that i intend to nomi...
5254,9.597990e+17,rasmussen just announced that my approval rati...,132165,30235,2018-03-02 14:40:00,2018-03-02 22:40:00,2018-03-02,22:40:00,22,2018,3,rasmussen just announced that my approval rati...
5255,9.576040e+17,somebody please inform jay-z that because of m...,202727,49876,2018-01-28 13:18:00,2018-01-28 21:18:00,2018-01-28,21:18:00,21,2018,1,somebody please inform jay-z that because of m...


### Pre Processing of Stock Data

In [ ]:
# Datasets are combined after pre processing! (else kernel crashes)
stock_price_2017 = pd.read_csv('yr2017.csv')
stock_price_2018 = pd.read_csv('yr2018.csv')

In [ ]:
# Drop unwanted columns 
stock_price_2017.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)
stock_price_2018.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)

In [ ]:
# Change to datetime objects
stock_price_2017['TIME_M'] = pd.to_datetime(stock_price_2017['TIME_M'], format='%H:%M:%S.%f')
stock_price_2018['TIME_M'] = pd.to_datetime(stock_price_2018['TIME_M'], format='%H:%M:%S.%f')

In [ ]:
# Remove seconds, milliseconds from data
stock_price_2017['TIME_M'] = stock_price_2017['TIME_M'].dt.floor('Min').dt.time
stock_price_2018['TIME_M'] = stock_price_2018['TIME_M'].dt.floor('Min').dt.time

In [ ]:
# Get minute to minute data (one entry for each minute)
stock_price_2017.drop_duplicates(subset=['TIME_M','DATE'],inplace=True)
stock_price_2018.drop_duplicates(subset=['TIME_M', 'DATE'],inplace=True)

In [ ]:
# Export to csv, will be used for preprocessing later!
stock_price = pd.concat([stock_price_2017,stock_price_2018], ignore_index=True, sort=False)
stock_price.to_csv("stock_price.csv", index=False, header=True)

In [ ]:
## Interpolation of missing values
idx = pd.date_range(stock_price.iloc[0, 0], stock_price.iloc[-1, 0], freq='1min')
stock_price.index = pd.DatetimeIndex(stock_price.loc[:, 'DATE_TIME_M'])
stock_price = stock_price.reindex(idx, fill_value='NaN')
stock_price['DATE_TIME_M'] = stock_price.index

In [ ]:
stock_price = stock_price.astype({'PRICE': 'float'})
stock_price['PRICE'].interpolate(method='linear', inplace=True)

In [ ]:
stock_price.to_csv("stock_price.csv", index=False, header=True)